이전에 RobustScaler와 LGBM을 결합했을 때 가장 좋은 성능이 나왔음을 확인할 수 있었다. 이번에는 SMOTE를 하고 안하고의 차이를 한 번 보려고 한다.

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('train.csv',index_col='id')
train

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,20.630780,20.437920,20.050756,20.020974,19.959646,20.948519,20.726709,20.311562,...,20.836400,20.536338,20.126561,20.102815,19.796722,20.783295,20.326879,20.014046,20.061940,19.862513
199987,GALAXY,362,25.283695,22.807243,21.069792,20.187531,19.595598,26.677671,22.818723,20.786120,...,28.895442,22.250489,20.091693,18.985791,18.803354,26.921903,21.952440,19.776321,18.990039,18.528306
199988,STAR_RED_DWARF,146,23.609826,21.902709,20.330747,18.457535,17.326771,24.320816,22.287946,20.711737,...,25.982378,22.132993,20.329624,18.507337,17.418521,24.503460,21.897599,20.420524,18.443126,17.331724


In [3]:
# RobustScaler
from sklearn.preprocessing import RobustScaler
train_for_scaling = train.iloc[:,2:] #type과 fiberID 제외하고 데이터 생성
robustScaler = RobustScaler()
print(robustScaler.fit(train_for_scaling))
train_robustscaled = robustScaler.transform(train_for_scaling) #robustscaling 적용
train_robustscaled = pd.DataFrame(train_robustscaled,columns=train_for_scaling.columns)
train_robustscaled = pd.concat([train[['type', 'fiberID']], train_robustscaled], axis=1) 
train_robustscaled #robustscaling과 type,fiberID 합치기

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)


,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,QSO,601,0.927836,0.623783,0.753702,0.916931,1.041252,0.639184,0.634486,0.805768,...,0.838327,0.638332,0.770905,0.825931,0.748549,0.859066,0.664478,0.744135,0.795289,0.978342
1,QSO,788,0.223298,0.328228,0.496221,0.713509,0.757221,0.341950,0.386929,0.531161,...,0.389795,0.412532,0.573234,0.655629,0.823214,0.398328,0.419522,0.539673,0.685895,0.774402
2,QSO,427,-1.204186,-1.296932,-1.124607,-1.010056,-0.801392,-1.200267,-1.222701,-1.046600,...,-0.980354,-0.985231,-0.837531,-0.656544,-0.455081,-0.949423,-0.987518,-0.838240,-0.662475,-0.453490
3,QSO,864,-0.032482,0.055586,0.191466,0.343934,0.346375,-0.004199,0.099931,0.233233,...,0.026301,0.140866,0.269585,0.356218,0.426423,0.133448,0.157305,0.250155,0.373217,0.466269
4,STAR_RED_DWARF,612,2.226460,1.288028,0.817437,0.198234,-0.090595,1.940455,1.433796,0.839643,...,2.154141,0.982669,0.836473,0.308756,0.067117,1.639432,1.247397,0.781377,0.282689,0.088207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,-0.095930,0.217909,0.241661,0.420093,0.548298,-0.041957,0.264872,0.270482,...,0.184161,0.328591,0.316947,0.474749,0.540180,0.138063,0.269970,0.291783,0.474791,0.565716
199987,GALAXY,362,1.759416,1.185329,0.654667,0.491704,0.400205,2.348040,1.107414,0.459206,...,3.344960,0.921627,0.305237,0.098497,0.209382,2.389373,0.833175,0.212136,0.116437,0.137232
199988,STAR_RED_DWARF,146,1.091962,0.815997,0.355138,-0.252104,-0.522744,1.364844,0.893648,0.429626,...,2.202441,0.880978,0.385142,-0.062662,-0.251776,1.502419,0.814175,0.427970,-0.066405,-0.247054
199989,GALAXY,85,0.119882,0.268249,0.233959,0.322353,0.377780,0.273255,0.161383,0.095711,...,-0.582843,-0.526213,-0.564692,-0.509540,-0.332802,-0.475946,-0.534496,-0.570515,-0.484095,-0.363270


In [4]:
#X,y 분류
X = train_robustscaled.drop(['type'], axis = 1)
y = train_robustscaled['type']

In [5]:
#SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_smote, y_smote = SMOTE(random_state=4).fit_sample(X, y)

In [6]:
import lightgbm
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=2000, max_depth=15,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)
lgbm_model.fit(X_smote,y_smote)

LGBMClassifier(bagging_fraction=0.9, boosting_type='gbdt', class_weight=None,
               colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.1, max_depth=15,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2000, n_jobs=-1, num_leaves=10, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.2, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [34]:
test = pd.read_csv('test.csv', index_col = 'id')
test

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
199991,251,23.817399,22.508963,20.981106,18.517316,17.076079,25.053890,23.167848,21.335901,18.835858,...,22.246697,22.796239,21.195315,18.584486,17.154284,25.391534,22.499435,21.011918,18.499341,17.091474
199992,386,22.806983,21.937111,20.335770,20.000512,19.527369,22.498565,22.186000,20.618879,20.301204,...,21.729831,21.837511,20.196128,19.967204,19.683671,22.475338,21.853442,20.173169,19.796757,19.567372
199993,232,21.024250,19.235669,18.304061,17.808608,17.380113,21.205546,19.439533,18.344433,17.909690,...,20.722629,18.710223,17.611851,17.158519,16.843986,20.579314,18.653338,17.562108,17.120529,16.708748
199994,557,20.503424,20.286261,20.197204,20.162419,20.059832,20.976132,20.611498,20.567262,20.479318,...,20.329269,20.385262,20.129157,20.206574,20.212342,20.479879,20.280943,20.150499,20.206221,20.092909
199995,75,24.244851,22.668237,21.239333,19.284777,18.235939,25.681860,22.935289,21.642456,19.624926,...,22.308298,22.957496,21.285033,19.299120,18.307526,25.489360,22.857290,21.191862,19.237964,18.280368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,389,19.765035,19.542406,19.439286,19.357957,19.357305,20.093308,19.827172,19.714033,19.686861,...,19.882728,19.549257,19.457564,19.303361,19.483389,19.792181,19.536518,19.432463,19.376445,19.354976
209996,247,19.960030,19.609379,19.325949,19.391290,19.258862,20.319914,19.928523,19.696262,19.763771,...,20.085454,19.635609,19.381924,19.460517,19.390865,20.066552,19.604800,19.334113,19.401240,19.159412
209997,941,21.221899,20.276182,20.090775,19.927208,19.766732,21.254454,20.551190,20.365800,20.200872,...,20.998020,20.315201,20.236725,20.058801,19.988363,21.045501,20.255485,20.141549,19.906014,19.953932


In [32]:
id_for_index = test['id']
id_for_index

0        199991
1        199992
2        199993
3        199994
4        199995
          ...  
10004    209995
10005    209996
10006    209997
10007    209998
10008    209999
Name: id, Length: 10009, dtype: int64

In [33]:
from sklearn.preprocessing import RobustScaler
test_for_scaling = test.iloc[:,2:] #fiberID 제외하고 데이터 생성
print(robustScaler.fit(test_for_scaling))
test_robustscaled = robustScaler.transform(test_for_scaling) #robustscaling 적용
test_robustscaled = pd.DataFrame(test_robustscaled,index = id_for_index, columns=test_for_scaling.columns)
test_robustscaled

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)


,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,
199991,1.182702,1.048251,0.626485,-0.220330,-0.618852,1.672996,1.233289,0.682468,-0.142070,-0.504366,0.734331,1.099892,0.676638,-0.033574,-0.338060,1.827431,1.014195,0.626034,-0.045304,-0.323146
199992,0.779387,0.817574,0.363304,0.413028,0.380063,0.607737,0.842195,0.395966,0.461799,0.530203,0.531978,0.771724,0.338297,0.431661,0.502168,0.757070,0.792720,0.343134,0.387349,0.473419
199993,0.067795,-0.272150,-0.465267,-0.522964,-0.494957,0.068706,-0.251791,-0.512839,-0.523744,-0.463003,0.137658,-0.298732,-0.536779,-0.513360,-0.441137,0.061153,-0.304420,-0.537547,-0.505100,-0.446280
199994,-0.140096,0.151644,0.306794,0.482166,0.597045,-0.026932,0.215032,0.375341,0.535200,0.628807,-0.016343,0.274626,0.315620,0.512201,0.677785,0.024657,0.253596,0.335487,0.523894,0.642499
199995,1.353323,1.112500,0.731795,0.107394,-0.146202,1.934783,1.140655,0.804959,0.183104,-0.043074,0.758448,1.155090,0.707018,0.206875,0.045031,1.863337,1.136884,0.686727,0.201007,0.059354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,-0.434830,-0.148417,-0.002300,0.138643,0.310761,-0.394962,-0.097385,0.034415,0.208628,0.442365,-0.191164,-0.011535,0.088208,0.208302,0.435637,-0.227756,-0.001626,0.093302,0.247187,0.405085
209996,-0.356996,-0.121401,-0.048521,0.152877,0.270645,-0.300494,-0.057014,0.027314,0.240323,0.281907,-0.111797,0.018023,0.062595,0.261179,0.404902,-0.127051,0.021784,0.060130,0.255455,0.342167
209997,0.146688,0.147578,0.263390,0.381726,0.477605,0.089095,0.191009,0.294843,0.420452,0.490485,0.245474,0.250644,0.352044,0.462480,0.603382,0.232262,0.244868,0.332468,0.423784,0.597786


In [35]:
test_robustscaled = pd.concat([test[['fiberID']], test_robustscaled], axis=1) 
test_robustscaled #standardscaling과 type,fiberID 합치기

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
199991,251,1.182702,1.048251,0.626485,-0.220330,-0.618852,1.672996,1.233289,0.682468,-0.142070,...,0.734331,1.099892,0.676638,-0.033574,-0.338060,1.827431,1.014195,0.626034,-0.045304,-0.323146
199992,386,0.779387,0.817574,0.363304,0.413028,0.380063,0.607737,0.842195,0.395966,0.461799,...,0.531978,0.771724,0.338297,0.431661,0.502168,0.757070,0.792720,0.343134,0.387349,0.473419
199993,232,0.067795,-0.272150,-0.465267,-0.522964,-0.494957,0.068706,-0.251791,-0.512839,-0.523744,...,0.137658,-0.298732,-0.536779,-0.513360,-0.441137,0.061153,-0.304420,-0.537547,-0.505100,-0.446280
199994,557,-0.140096,0.151644,0.306794,0.482166,0.597045,-0.026932,0.215032,0.375341,0.535200,...,-0.016343,0.274626,0.315620,0.512201,0.677785,0.024657,0.253596,0.335487,0.523894,0.642499
199995,75,1.353323,1.112500,0.731795,0.107394,-0.146202,1.934783,1.140655,0.804959,0.183104,...,0.758448,1.155090,0.707018,0.206875,0.045031,1.863337,1.136884,0.686727,0.201007,0.059354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,389,-0.434830,-0.148417,-0.002300,0.138643,0.310761,-0.394962,-0.097385,0.034415,0.208628,...,-0.191164,-0.011535,0.088208,0.208302,0.435637,-0.227756,-0.001626,0.093302,0.247187,0.405085
209996,247,-0.356996,-0.121401,-0.048521,0.152877,0.270645,-0.300494,-0.057014,0.027314,0.240323,...,-0.111797,0.018023,0.062595,0.261179,0.404902,-0.127051,0.021784,0.060130,0.255455,0.342167
209997,941,0.146688,0.147578,0.263390,0.381726,0.477605,0.089095,0.191009,0.294843,0.420452,...,0.245474,0.250644,0.352044,0.462480,0.603382,0.232262,0.244868,0.332468,0.423784,0.597786


In [36]:
test_pred_prob_lgbm_robustscaled_smote = lgbm_model.predict_proba(test_robustscaled)
test_pred_prob_lgbm_robustscaled_smote

array([[1.01188853e-06, 7.45653427e-04, 1.97054184e-09, ...,
        7.24403396e-01, 9.97829786e-09, 5.09683928e-10],
       [4.16341106e-04, 1.41400124e-01, 8.56851148e-08, ...,
        1.34595771e-09, 9.81448190e-10, 6.60766229e-07],
       [9.37602428e-01, 2.41361425e-03, 4.87515465e-10, ...,
        3.26301140e-07, 1.73808626e-12, 1.93818436e-04],
       ...,
       [9.27600895e-10, 9.99977434e-01, 1.60523413e-10, ...,
        1.80259605e-14, 7.27278620e-19, 3.67326266e-11],
       [4.37953918e-09, 9.99997309e-01, 1.90433615e-10, ...,
        2.42002478e-10, 3.46223092e-15, 7.41086218e-14],
       [4.80571859e-06, 1.54458677e-04, 1.86182284e-08, ...,
        5.55389562e-09, 1.11951544e-09, 1.23628437e-03]])

In [37]:
result = pd.DataFrame(data=test_pred_prob_lgbm_robustscaled_smote, index=id_for_index, columns=lgbm_model.classes_)
result.head()

,GALAXY,QSO,REDDEN_STD,ROSAT_D,SERENDIPITY_BLUE,SERENDIPITY_DISTANT,SERENDIPITY_FIRST,SERENDIPITY_MANUAL,SERENDIPITY_RED,SKY,SPECTROPHOTO_STD,STAR_BHB,STAR_BROWN_DWARF,STAR_CARBON,STAR_CATY_VAR,STAR_PN,STAR_RED_DWARF,STAR_SUB_DWARF,STAR_WHITE_DWARF
id,,,,,,,,,,,,,,,,,,,
199991,1.011889e-06,0.000746,1.970542e-09,0.001726,4.275192e-06,6.909108e-08,2.411846e-06,0.003864,2.555725e-01,2.844430e-10,1.189026e-07,8.725925e-10,1.368016e-02,8.668850e-08,7.355502e-08,3.648141e-17,7.244034e-01,9.978298e-09,5.096839e-10
199992,4.163411e-04,0.141400,8.568511e-08,0.486675,5.277143e-06,1.182180e-04,3.712275e-01,0.000143,7.477359e-06,3.469718e-10,3.826963e-08,4.927408e-06,2.088305e-10,1.004562e-06,7.158873e-08,3.849191e-11,1.345958e-09,9.814482e-10,6.607662e-07
199993,9.376024e-01,0.002414,4.875155e-10,0.038822,1.014370e-05,8.406232e-08,3.426224e-04,0.020609,2.720090e-07,2.237232e-10,7.394245e-07,2.483235e-08,3.265131e-09,4.159353e-06,1.055898e-06,2.082797e-13,3.263011e-07,1.738086e-12,1.938184e-04
199994,3.608836e-06,0.033211,3.586226e-07,0.034812,8.898007e-01,2.031713e-03,3.996570e-02,0.000167,5.144087e-07,5.504952e-10,1.208449e-07,1.641290e-07,7.643117e-12,8.695719e-08,5.092485e-07,2.430579e-09,5.342607e-10,9.796859e-15,7.051159e-06
199995,2.620024e-07,0.000019,6.621066e-08,0.000691,1.145838e-08,4.560226e-09,2.355288e-07,0.000004,4.429530e-05,8.655323e-12,1.328381e-09,2.606975e-09,1.916930e-06,2.888149e-08,6.893025e-11,5.953698e-18,9.992393e-01,1.128410e-11,2.067661e-09


In [38]:
sample = pd.read_csv("sample_submission.csv")
col_order = sample.columns[1:]
result = result[col_order]
result

,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
id,,,,,,,,,,,,,,,,,,,
199991,5.096839e-10,7.355502e-08,1.368016e-02,2.555725e-01,1.970542e-09,8.725925e-10,1.011889e-06,6.909108e-08,0.000746,2.844430e-10,7.244034e-01,1.726015e-03,3.648141e-17,2.411846e-06,8.668850e-08,1.189026e-07,9.978298e-09,3.864241e-03,4.275192e-06
199992,6.607662e-07,7.158873e-08,2.088305e-10,7.477359e-06,8.568511e-08,4.927408e-06,4.163411e-04,1.182180e-04,0.141400,3.469718e-10,1.345958e-09,4.866752e-01,3.849191e-11,3.712275e-01,1.004562e-06,3.826963e-08,9.814482e-10,1.430968e-04,5.277143e-06
199993,1.938184e-04,1.055898e-06,3.265131e-09,2.720090e-07,4.875155e-10,2.483235e-08,9.376024e-01,8.406232e-08,0.002414,2.237232e-10,3.263011e-07,3.882172e-02,2.082797e-13,3.426224e-04,4.159353e-06,7.394245e-07,1.738086e-12,2.060899e-02,1.014370e-05
199994,7.051159e-06,5.092485e-07,7.643117e-12,5.144087e-07,3.586226e-07,1.641290e-07,3.608836e-06,2.031713e-03,0.033211,5.504952e-10,5.342607e-10,3.481165e-02,2.430579e-09,3.996570e-02,8.695719e-08,1.208449e-07,9.796859e-15,1.668973e-04,8.898007e-01
199995,2.067661e-09,6.893025e-11,1.916930e-06,4.429530e-05,6.621066e-08,2.606975e-09,2.620024e-07,4.560226e-09,0.000019,8.655323e-12,9.992393e-01,6.908882e-04,5.953698e-18,2.355288e-07,2.888149e-08,1.328381e-09,1.128410e-11,3.709791e-06,1.145838e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,8.132114e-02,2.630523e-02,2.898906e-13,6.922766e-07,2.584994e-07,3.440151e-05,1.289564e-05,3.887542e-03,0.045576,5.545919e-10,1.077490e-11,1.703510e-01,2.607453e-11,2.945041e-02,1.305310e-10,7.138723e-08,1.145704e-17,3.636836e-04,6.426965e-01
209996,1.167898e-01,3.219596e-02,4.969472e-12,1.629053e-07,6.874822e-07,1.507836e-05,3.274418e-05,5.292296e-02,0.095064,7.130192e-10,1.560813e-11,2.079630e-01,7.672266e-10,2.911492e-01,2.224656e-08,2.546218e-07,1.218449e-16,3.879680e-04,2.034779e-01
209997,3.673263e-11,4.602066e-10,1.411395e-16,3.173944e-11,1.605234e-10,2.967629e-09,9.276009e-10,1.382698e-08,0.999977,6.668491e-14,1.802596e-14,9.832117e-06,3.270626e-16,1.218750e-05,2.827769e-13,8.653828e-13,7.272786e-19,5.708371e-11,5.282888e-07


In [ ]:
result.to_csv('robustscaler_smote_lgbm.csv',sep=',')